# The Notebook Perform Author and deploy a tool-calling LangGraph agent using Mosaic AI Agent Framework: 

- Author a VectorSearch Tool-Calling LangGraph agent wrapped with `ChatAgent`
- Log and deploy the agent

To learn more about authoring an agent using Mosaic AI Agent Framework, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/author-agent) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/create-chat-model)).

In [0]:
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.



## Define the agent in code
Define the agent code in a single cell below. This lets you easily write the agent code to a local Python file, using the `%%writefile` magic command, for subsequent logging and deployment.

#### Agent tools
This agent code adds the built-in Unity Catalog function `system.ai.python_exec` to the agent. The agent code also includes commented-out sample code for adding a vector search index to perform unstructured data retrieval.

For more examples of tools to add to your agent, see Databricks documentation ([AWS](https://docs.databricks.com/aws/generative-ai/agent-framework/agent-tool) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-framework/agent-tool))

#### Wrap the LangGraph agent using the `ChatAgent` interface

For compatibility with Databricks AI features, the `LangGraphChatAgent` class implements the `ChatAgent` interface to wrap the LangGraph agent. This example uses the provided convenience APIs [`ChatAgentState`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentState) and [`ChatAgentToolNode`](https://mlflow.org/docs/latest/python_api/mlflow.langchain.html#mlflow.langchain.chat_agent_langgraph.ChatAgentToolNode) for ease of use.

Databricks recommends using `ChatAgent` as it simplifies authoring multi-turn conversational agents using an open source standard. See MLflow's [ChatAgent documentation](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent).



In [0]:
#%%writefile policy_doc_rag_agent.py
from typing import Any, Generator, Optional, Sequence, Union
import uuid
import mlflow
from databricks_langchain import ChatDatabricks, VectorSearchRetrieverTool
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)
from langchain_core.language_models import LanguageModelLike
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.tools import BaseTool
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

mlflow.langchain.autolog()

client = DatabricksFunctionClient()
set_uc_function_client(client)

############################################
# Define your LLM endpoint and system prompt
############################################
# TODO: Replace with your model serving endpoint
LLM_ENDPOINT_NAME = "databricks-meta-llama-3-3-70b-instruct"
llm = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME)

# TODO: Update with your system prompt
system_prompt = """You are a trusted insurance company policy information assistant. 
                Your task is to answer user queries about insurance policy.
                Always use the policy_doc_search tool to retrieve relevant information before answering.
                Provide concise and accurate responses based on the retrieved documentation.
                If you do not know the answer to a question, you truthfully say you do not know."""

###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/en/generative-ai/agent-framework/agent-tool.html
###############################################################################
CATALOG = "fins_genai"
SCHEMA = "call_center"

# Use Databricks vector search indexes as tools
# See https://docs.databricks.com/en/generative-ai/agent-framework/unstructured-retrieval-tools.html
# for details
tools = []
vector_search_tools = [
        VectorSearchRetrieverTool(
        index_name="fins_genai.call_center.policy_docs_chunked_files_vs_index",
        tool_name="policy_doc_search",
        tool_description="Vector Search Retriever tool for insurance policy documents",
    )
]
tools.extend(vector_search_tools)

#####################
## Define agent logic
#####################


def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolNode, Sequence[BaseTool]],
    system_prompt: Optional[str] = None,
) -> CompiledGraph:
    model = model.bind_tools(tools)
    memory = MemorySaver()

    # Define the function that determines which node to go to
    def should_continue(state: ChatAgentState):
        messages = state["messages"]
        last_message = messages[-1]
        # If there are function calls, continue. else, end
        if last_message.get("tool_calls"):
            return "continue"
        else:
            return "end"

    if system_prompt:
        preprocessor = RunnableLambda(
            lambda state: [{"role": "system", "content": system_prompt}]
            + state["messages"]
        )
    else:
        preprocessor = RunnableLambda(lambda state: state["messages"])
    model_runnable = preprocessor | model

    # def chat_history_cutoff(state: ChatAgentState):
    #     state['messages'] = state['messages'][-5:]
    #     return state

    def call_model(
        state: ChatAgentState,
        config: RunnableConfig,
    ):
        #state = chat_history_cutoff(state)
        response = model_runnable.invoke(state, config)

        return {"messages": [response]}

    workflow = StateGraph(ChatAgentState)

    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", ChatAgentToolNode(tools))

    workflow.set_entry_point("agent")
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "continue": "tools",
            "end": END,
        },
    )
    workflow.add_edge("tools", "agent")

    return workflow.compile(checkpointer=memory)


class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph):
        self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}

        messages = []
        config = {"configurable": {"thread_id": "1"}}
        for event in self.agent.stream(request, config, stream_mode="updates"):
            for node_data in event.values():
                messages.extend(
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                )
        return ChatAgentResponse(messages=messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {"messages": self._convert_messages_to_dict(messages)}
        config = {"configurable": {"thread_id": "1"}}
        for event in self.agent.stream(request, config, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )


# Create the agent object, and specify it as the agent object to use when
# loading the agent back for inference via mlflow.models.set_model()
agent = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent)
#mlflow.models.set_model(AGENT)

/home/spark-7a32c863-6726-47a3-942c-38/.ipykernel/35100/command-1332008936305341-4250846646:6: DeprecationWarning: Imports from this module are deprecated and will be removed in a future release. Please update the code to import directly from databricks_langchain.

For example, replace imports like: `from databricks_langchain.uc_ai import UCFunctionToolkit`
with: `from databricks_langchain import UCFunctionToolkit`
  from databricks_langchain.uc_ai import (
/local_disk0/.ephemeral_nfs/envs/pythonEnv-7a32c863-6726-47a3-942c-38c8a5214a1c/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
/home/spark-7a32c863-6726-47a3-942c-38/.ipykernel/35100/command-1332008936305341-4250846646:38: DeprecationWarning: Currently, temperat

In [0]:
AGENT.predict({"messages": [{"role": "user", "content": "Hello, my name is Qian"}]})

ChatAgentResponse(messages=[ChatAgentMessage(role='assistant', content="Hello Qian, it's nice to meet you. I'm here to help answer any questions you may have about insurance policies. Is there something specific you'd like to know or discuss?", name=None, id='run-7a3bb10c-d92d-4cf7-8f19-2aef9de3b189-0', tool_calls=None, tool_call_id=None, attachments=None)], finish_reason=None, custom_outputs=None, usage=None)

Trace(request_id=tr-e5f3dbaee6394a75914db240e89ab012)

In [0]:
AGENT.predict({"messages": [{"role": "user", "content": "what is my name?"}]})

ChatAgentResponse(messages=[ChatAgentMessage(role='assistant', content='Your name is Qian.', name=None, id='run-ecf67276-9b43-46a3-8f0f-68f91d984da9-0', tool_calls=None, tool_call_id=None, attachments=None)], finish_reason=None, custom_outputs=None, usage=None)

Trace(request_id=tr-70287a85e12d403a957f62ecdfe50ca3)

## Test the agent

Interact with the agent to test its output and tool-calling abilities. Since this notebook called `mlflow.langchain.autolog()`, you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from policy_doc_rag_agent import AGENT

AGENT.predict({"messages": [{"role": "user", "content": "Hello, my name is Qian"}]})

/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


ChatAgentResponse(messages=[ChatAgentMessage(role='assistant', content="Hello Qian, it's nice to meet you. I'm here to help answer any questions you may have about insurance policies. What's on your mind? Do you have a specific question or topic you'd like to discuss?", name=None, id='run-fd589e94-d7fc-4b36-ba11-46260564ea8c-0', tool_calls=None, tool_call_id=None, attachments=None)], finish_reason=None, custom_outputs=None, usage=None)

Trace(request_id=tr-1c5707aad3ba42a89d5b2f336c608dbe)

## Log the agent as an MLflow model

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

### Enable automatic authentication for Databricks resources
For the most common Databricks resource types, Databricks supports and recommends declaring resource dependencies for the agent upfront during logging. This enables automatic authentication passthrough when you deploy the agent. With automatic authentication passthrough, Databricks automatically provisions, rotates, and manages short-lived credentials to securely access these resource dependencies from within the agent endpoint.

To enable automatic authentication, specify the dependent Databricks resources when calling `mlflow.pyfunc.log_model().`

  - **TODO**: If your Unity Catalog tool queries a [vector search index](docs link) or leverages [external functions](docs link), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See docs ([AWS](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-framework/log-agent#resources)).



In [0]:
import mlflow
from policy_doc_rag_agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))


with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="langgraph_policy_rag_agent",
        python_model="policy_doc_rag_agent.py",
        pip_requirements=[
            "mlflow",
            "langchain",
            "langgraph==0.3.4",
            "databricks-langchain",
            "unitycatalog-langchain[databricks]",
            "pydantic",
        ],
        resources=resources,
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/03/26 21:23:00 INFO mlflow.pyfunc: Predicting on input example to validate output
/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

## Evaluate the agent with Agent Evaluation

Use Mosaic AI Agent Evaluation to evalaute the agent's responses based on expected responses and other evaluation criteria. Use the evaluation criteria you specify to guide iterations, using MLflow to track the computed quality metrics.
See Databricks documentation ([AWS]((https://docs.databricks.com/aws/generative-ai/agent-evaluation) | [Azure](https://learn.microsoft.com/azure/databricks/generative-ai/agent-evaluation/)).


To evaluate your tool calls, add custom metrics. See Databricks documentation ([AWS](https://docs.databricks.com/en/generative-ai/agent-evaluation/custom-metrics.html#evaluating-tool-calls) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/generative-ai/agent-evaluation/custom-metrics#evaluating-tool-calls)).

In [0]:
import pandas as pd

eval_examples = [
    {
        "request": {"messages": [{"role": "user", "content": "Car accident claim policy"}]},
        "expected_response": """Based on the provided policy documents, in the event of a car accident, the insurance policy will cover damages sustained by anyone else as a result of bodily injury. The policy will pay for damages arising out of bodily injury to two or more persons in any one motor vehicle accident, with a maximum limit that will not be increased even if there are multiple claims or vehicles involved. The policy also provides for funeral service expenses benefits in the event of an insured person's death as a direct result of a covered auto accident.
        To make a claim, the insured person or their representative must provide written proof of claim, including all details reasonably required by the insurance company. The insurance company may also require the insured person to submit to questioning under oath and sign the transcript.
        It's important to note that the policy has certain exclusions, such as death sustained in the course of an occupation or due to suicide, and that the insurance company has the right to investigate and settle claims. The policy also provides for the payment of certain expenses, such as court costs and interest on damages awarded.""",
    }
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)


request,expected_response
"List(List(List(Car accident claim policy, user)))","Based on the provided policy documents, in the event of a car accident, the insurance policy will cover damages sustained by anyone else as a result of bodily injury. The policy will pay for damages arising out of bodily injury to two or more persons in any one motor vehicle accident, with a maximum limit that will not be increased even if there are multiple claims or vehicles involved. The policy also provides for funeral service expenses benefits in the event of an insured person's death as a direct result of a covered auto accident. To make a claim, the insured person or their representative must provide written proof of claim, including all details reasonably required by the insurance company. The insurance company may also require the insured person to submit to questioning under oath and sign the transcript. It's important to note that the policy has certain exclusions, such as death sustained in the course of an occupation or due to suicide, and that the insurance company has the right to investigate and settle claims. The policy also provides for the payment of certain expenses, such as court costs and interest on damages awarded."


In [0]:
import mlflow

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/langgraph_policy_rag_agent",
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/03/26 21:23:12 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?]

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results: 0/1 pass

request_id request response retrieved_context trace tool_calls expected_response response/overall_assessment/rating response/overall_assessment/rationale response/llm_judged/safety/rating response/llm_judged/safety/rationale response/llm_judged/correctness/rating response/llm_judged/correctness/rationale response/llm_judged/groundedness/rating response/llm_judged/groundedness/rationale retrieval/llm_judged/context_sufficiency/rating retrieval/llm_judged/context_sufficiency/rationale agent/total_token_count agent/total_input_token_count agent/total_output_token_count agent/latency_seconds 71aac4a42fefdbd9bfb8b4706a11a0fa6843b3d93ca693dd29d8d4837cf4a1d2 List(List(List(Car accident claim policy, user))) List(List(List(, run-422936f5-7129-408b-ae8f-e708dfe17f91-0, null, assistant, null, List(List(List({"query": "car accident claim policy"}, policy_doc_search), call_d1d95d6b-1f68-4bb1-9a40-15ba42247035, function))), List([Document(metadata={'chunk_id': '10403fe747e34f845114d226b570915f'}, page_content='MOTOR VEHICLE ACCIDENT, INCLUDING\nDAMAGES SUSTAINED BYANYONE ELSE ASA\nRESULT OFTHAT BODILY INJURY .\n2."EACH ACCIDENT" ISTHEMAXIMUM THAT WE\nWILL PAY FORDAMAGES ARISING OUT OF\nBODILY INJURY TOTWO ORMORE PERSONS\nINANY ONE MOTOR VEHICLE ACCIDENT. THIS\nLIMIT ISSUBJECT TOTHELIMIT FOR "EACH\nPERSON".\nPage 11\nS\nA\nM\nP\nL\nE\nD\nO\nC\nU\nM\nE\nN\nTAutosure Insurance Company\nTHESE LIMITS ARE THEMAXIMUM ALLSTATE\nWILL PAY FORANY ONE MOTOR VEHICLE\nACCIDENT REGARDLESS OFTHENUMBER OF:\n1.CLAIMS MADE;\n2.VEHICLES ORPERSONS SHOWN ONTHE\nPOLICY DECLARATIONS; OR\n3.VEHICLES INVOLVED INTHEACCIDENT.\nDAMAGES PAYABLE WILL BEREDUCED BY\n1.ALLAMOUNTS PAID BYTHEOWNER OR\nOPERATOR OFTHEUNINSURED AUTO OR\nANYONE ELSE RESPONSIBLE. THIS INCLUDES\nALLSUMS PAID UNDER THEBODILY INJURY\nLIABILITY COVERAGE OFTHIS ORANY OTHER\nAUTO POLICY.\n2.ALLAMOUNTS PAYABLE UNDER ANY\nWORKERS COMPENSATION LAW, DISABILITY\nBENEFITS LAW, ORSIMILAR LAW,\nAUTOMOBILE MEDICAL PAYMENTS, ORANY\nSIMILAR AUTOMOBILE MEDICAL PAYMENTS\nCOVERAGE.\nWEARE NOT OBLIGATED TOMAKE ANY PAYMENT\nFORBODILY INJURY UNDER THIS COVERAGE\nWHICH ARISES OUT OFTHEUSE OFAN\nUNDERINSURED MOTOR VEHICLE UNTIL AFTER\nTHELIMITS OFLIABILITY FOR ALLLIABILITY\nPROTECTION INEFFECT AND APPLICABLE ATTHE\nTIME OFTHEACCIDENT HAVE BEEN EXHAUSTED\nBYPAYMENT OFJUDGMENTS ORSETTLEMENTS.\nIFTHERE ISOTHER INSURANCE\nIFTHEINSURED PERSON WAS IN,ON,GETTING\nINTO OROUT OFAVEHICLE YOU DONOT OWN\nWHICH ISINSURED FOR THIS COVERAGE UNDER\nANOTHER POLICY, THIS COVERAGE WILL BE\nEXCESS. THIS MEANS THAT WHEN THEINSURED\nPERSON ISLEGALLY ENTITLED TORECOVER\nDAMAGES INEXCESS OFTHEOTHER POLICY\nLIMIT, WEWILL PAY UPTOYOUR POLICY LIMIT,\nBUT ONLY AFTER THEOTHER INSURANCE HAS\nBEEN EXHAUSTED.\nIFMORE THAN ONE POLICY APPLIES TOTHE\nACCIDENT ONAPRIMARY BASIS, THETOTAL\nBENEFITS PAYABLE TOANY ONE PERSON WILL\nNOT EXCEED THEMAXIMUM BENEFITS PAYABLE\nBYTHEPOLICY WITH THEHIGHEST LIMIT FOR\nUNINSURED MOTORISTS COVERAGE. WEWILL\nBEAR OUR PROPORTIONATE SHARE WITH OTHER\nUNINSURED MOTORISTS BENEFITS. THIS APPLIESNOMATTER HOW MANY AUTOS ORAUTO\nPOLICIES MAY BEINVOLVED WHETHER WRITTEN\nBYALLSTATE ORANOTHER COMPANY.\nProof OfClaim; Medical Reports\nAssoon aspossible, youoranyother person making\nclaim must giveuswritten proof ofclaim, including\nalldetails reasonably required byustodetermine the\namounts payable. Wemay alsorequire anyperson\nmaking claim tosubmit toquestioning under oath\nandsign thetranscript.\nTheinsured person may berequired totakemedical\nexaminations byphysicians selected byus,asoften\naswereasonably require. Theinsured person orthat\nperson\'s representative must authorize ustoobtain\nmedical reports andcopies ofrecords.\nAssistance AndCooperation\nWemay require theinsured person totakeproper\naction topreserve allrights torecover damages from\nanyone responsible forthebodily injury .\nTrust Agreement\nWhen wepayanyperson under thiscoverage:\n1.weareentitled torepayment ofamounts paid by\nusandrelated collection expenses outo

## Pre-deployment agent validation
Before registering and deploying the agent, perform pre-deployment checks using the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See Databricks documentation ([AWS](https://docs.databricks.com/en/machine-learning/model-serving/model-serving-debug.html#validate-inputs) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/machine-learning/model-serving/model-serving-debug#before-model-deployment-validation-checks)).

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/langgraph_policy_rag_agent",
    input_data={"messages": [{"role": "user", "content": "Car accident claim policy"}]},
)

2025/03/26 21:23:35 INFO mlflow.models.python_api: It is highly recommended to use `uv` as the environment manager for predicting with MLflow models as its performance is significantly better than other environment managers. Run `pip install uv` to install uv. See https://docs.astral.sh/uv/getting-started/installation for other installation methods.
2025/03/26 21:23:36 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/03/26 21:23:37 INFO mlflow.utils.virtualenv: Environment /tmp/virtualenv_envs/mlflow-f82126561f76e2582d8398ffda5900c464a2e976 already exists
2025/03/26 21:23:37 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-f82126561f76e2582d8398ffda5900c464a2e976/bin/activate && python -c ""']'
2025/03/26 21:23:37 INFO mlflow.utils.environment: === Running command '['bash', '-c', 'source /tmp/virtualenv_envs/mlflow-f82126561f76e2582d8398ffda5900c464a2e976/bin/activate && python /local_di

{"messages": [{"role": "assistant", "content": "", "id": "run-b5c5038b-d275-4002-badd-e1a0ff623440-0", "tool_calls": [{"id": "call_04e34463-aada-4f9d-8152-6911d15c4ed4", "type": "function", "function": {"name": "policy_doc_search", "arguments": "{\"query\": \"car accident claim policy\"}"}}]}, {"role": "tool", "content": "[Document(metadata={'chunk_id': '10403fe747e34f845114d226b570915f'}, page_content='MOTOR VEHICLE ACCIDENT, INCLUDING\\nDAMAGES SUSTAINED BYANYONE ELSE ASA\\nRESULT OFTHAT BODILY INJURY .\\n2.\"EACH ACCIDENT\" ISTHEMAXIMUM THAT WE\\nWILL PAY FORDAMAGES ARISING OUT OF\\nBODILY INJURY TOTWO ORMORE PERSONS\\nINANY ONE MOTOR VEHICLE ACCIDENT. THIS\\nLIMIT ISSUBJECT TOTHELIMIT FOR \"EACH\\nPERSON\".\\nPage 11\\nS\\nA\\nM\\nP\\nL\\nE\\nD\\nO\\nC\\nU\\nM\\nE\\nN\\nTAutosure Insurance Company\\nTHESE LIMITS ARE THEMAXIMUM ALLSTATE\\nWILL PAY FORANY ONE MOTOR VEHICLE\\nACCIDENT REGARDLESS OFTHENUMBER OF:\\n1.CLAIMS MADE;\\n2.VEHICLES ORPERSONS SHOWN ONTHE\\nPOLICY DECLARATIONS;

## Register the model to Unity Catalog

Before you deploy the agent, you must register the agent to Unity Catalog.

- **TODO** Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "fins_genai"
schema = "call_center"
model_name = "langgraph_call_center_vs"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

Registered model 'fins_genai.call_center.langgraph_call_center_vs' already exists. Creating a new version of this model...
Created version '5' of model 'fins_genai.call_center.langgraph_call_center_vs'.


## Deploy the agent

In [0]:
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "docs"})

Traceback (most recent call last):
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/agents/deployments.py", line 755, in deploy
    deployment_info = _create_review_app_and_update_url(deployment_info)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/agents/deployments.py", line 949, in _create_review_app_and_update_url
    my_review_app = review_app.get_review_app(experiment_id)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/rag_eval/review_app/api.py", line 95, in get_review_app
    return _create_review_app(client, experiment_id)
  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-a867763e-5183-4f76-b386-bb128e907207/lib/python3.10/site-packages/databricks/rag_eval/review_app/api.py", line 68, in _create_review_app
    return client.create_review_app


    Deployment of fins_genai.call_center.langgraph_call_center_vs version 5 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://adb-984752964297111.11.azuredatabricks.net/ml/endpoints/agents_fins_genai-call_center-langgraph_call_center_vs
    Review App: https://adb-984752964297111.11.azuredatabricks.net/ml/review/fins_genai.call_center.langgraph_call_center_vs/5?o=984752964297111


Deployment(model_name='fins_genai.call_center.langgraph_call_center_vs', model_version='5', endpoint_name='agents_fins_genai-call_center-langgraph_call_center_vs', served_entity_name='fins_genai-call_center-langgraph_call_center_vs_5', query_endpoint='https://adb-984752964297111.11.azuredatabricks.net/serving-endpoints/agents_fins_genai-call_center-langgraph_call_center_vs/served-models/fins_genai-call_center-langgraph_call_center_vs_5/invocations', endpoint_url='https://adb-984752964297111.11.azuredatabricks.net/ml/endpoints/agents_fins_genai-call_center-langgraph_call_center_vs', review_app_url='https://adb-984752964297111.11.azuredatabricks.net/ml/review/fins_genai.call_center.langgraph_call_center_vs/5?o=984752964297111')